<a href="https://colab.research.google.com/github/jaredzrks0/DSAN5400_final_project/blob/main/DSAN_5400_Final_News_Recommender_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Change so file pushes

In [2]:
import os

In [3]:
# Install ColBERT and add to path

!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git > /dev/null 2>&1
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory


In [4]:
# Download the ColBERT Checkpoint
!wget "https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz"
!mkdir -p checkpoints  # Create the 'checkpoints' directory if it doesn't exist
!tar -xvzf colbertv2.0.tar.gz -C checkpoints

--2024-12-09 23:49:09--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.10MB/s    in 75s     

2024-12-09 23:50:24 (5.19 MB/s) - ‘colbertv2.0.tar.gz’ saved [405924985/405924985]

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin


In [5]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip > /dev/null 2>&1
    !pip install fsspec==2024.9.0 > /dev/null 2>&1
    !pip install -e ColBERT/['faiss-gpu','torch'] > /dev/null 2>&1
    !pip install --upgrade torch torchvision torchaudio > /dev/null 2>&1
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [6]:
# ColBERT Imports
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [3]:
import pandas as pd
import logging

article_df = pd.read_csv('../article_scraping/data/articles_with_text.csv')
article_df['doc_id'] = range(len(article_df))

articles_subset = article_df[['doc_id', 'article_text_raw', 'url', 'titles']]
articles = articles_subset['article_text_raw'].tolist()
metadata = articles_subset[['doc_id', 'url', 'titles']].set_index('doc_id').to_dict(orient='index')


In [9]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'dsan5400_project_nbits={nbits}'

In [10]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='DSAN5400')):
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)
    indexer = Indexer(checkpoint=checkpoint, config=config)
    logging.info(f"Starting indexing process for {len(articles)} articles...")
    indexer.index(name=index_name, collection=articles, overwrite=True)
    logging.info("Indexing complete.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[Dec 09, 23:58:12] #> Creating directory /content/experiments/DSAN5400/indexes/dsan5400_project_nbits=2 


#> Starting...
#> Joined...
